<a href="https://colab.research.google.com/github/ev1025/GA_LOG_ANALYST/blob/main/Hits%EC%BB%AC%EB%9F%BC_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# hits컬럼 분석

In [ ]:
pip install ipywidgets # tqdm

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [ ]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
import os
import csv
import re
import ast
import copy
from tqdm.notebook import tqdm
from datetime import datetime


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# train_v2가 있는 경로로 이동
os.chdir('C:\\Users\\ENVY\\Desktop\\GA')

## hits컬럼

In [ ]:
# totals의 hits 값 가져오기위해 데이터 읽어오기
df_index = pd.read_csv('train_v2.csv',
                        usecols=['totals'],  # hits(총 조회수 정보 추출)
                        converters={'totals':ast.literal_eval}) # str형식을 json형식으로 바꿔주기

# json을 columns : 값 형식으로 바꿔주기
df_index_df = json_normalize(df_index['totals']) 
df_index_df['hits'] = df_index_df['hits'].astype(int)

# hits가 100이상이 아닌 컬럼을 뽑아준다.
hit_100_index = np.array(df_index_df[~(df_index_df['hits']>100)].index)
# read_csv의 skiprows는 인덱스에 +1을 해줘야하기 때문에 빠른 계산을 위해 np로 각 index에 +1
hit_100_index +=1

In [ ]:
# hits가 100개 이상인 컬럼은 1541개
len(df_index_df)-len(hit_100_index)

1541

In [ ]:
# totals의 hits(하루 총 조회수)가 100 이상인 고객의 hits컬럼 추출(고유아이디 포함)
df = pd.read_csv('train_v2.csv',
                  header=0, 
                  skiprows=hit_100_index, 
                  usecols=['fullVisitorId','visitStartTime','hits'],
                  dtype = {'fullVisitorId':str},
                  converters={'hits': ast.literal_eval})
df.head()

,fullVisitorId,hits,visitStartTime
0,309482894121265066,"[{'hitNumber': '1', 'time': '0', 'hour': '8', ...",1508166548
1,9291001063810206156,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",1508190255
2,4459364070259865039,"[{'hitNumber': '1', 'time': '0', 'hour': '13',...",1472847498
3,5911716885988565842,"[{'hitNumber': '1', 'time': '0', 'hour': '19',...",1472870925
4,2940846817699257423,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",1472851842


In [ ]:
# 값을 넣어줄 빈 데이터프레임 생성, json을 DataFrame화
empty_data = pd.DataFrame()
hit_normal = json_normalize(df['hits'])
hits_max = df_index_df['hits'].max()

# 각 고객별 hits정보(json)를 풀어서 세로로 붙인다.
for i in tqdm(range(hits_max)):
    temp = json_normalize(hit_normal[i])
    empty_data = pd.concat([empty_data,temp],axis=0)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# 각 값에 고유아이디와 시간을 넣어준다.
hits_100 = pd.merge(df[['fullVisitorId','visitStartTime']],empty_data, how='outer',left_index=True,right_index=True)
hits_100 = hits_100.fillna({'fullVisitorId': df['fullVisitorId'].values, 'visitStartTime':df['visitStartTime'].values})
hits_100['visitStartTime'] =  hits_100['visitStartTime'].apply(lambda x : datetime.utcfromtimestamp(x))
hits_100.head()

,fullVisitorId,visitStartTime,hitNumber,time,hour,minute,isInteraction,isEntrance,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,promotionActionInfo.promoIsView,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,referer,transaction.currencyCode,item.currencyCode,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,contentGroup.contentGroupUniqueViews1,eCommerceAction.option,promotionActionInfo.promoIsClick,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,transaction.transactionId,transaction.transactionRevenue,transaction.transactionTax,transaction.transactionShipping,transaction.affiliation,transaction.localTransactionRevenue,transaction.localTransactionTax,transaction.localTransactionShipping,item.transactionId,transaction.transactionCoupon,isExit
0,309482894121265066,2017-10-16 15:09:08,1,0,8,9,True,True,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,,,,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,True,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(entrance),(entrance),(entrance),(entrance),(entrance),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,2,7662,8,9,True,NaN,[],[],[],[],[],[],PAGE,web,[],/signin.html,shop.googlemerchandisestore.com,The Google Merchandise Store - Log In,/signin.html,,,,shop.googlemerchandisestore.com/signin.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,NaN,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,3,17130,8,9,True,NaN,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/store.html,shop.googlemerchandisestore.com,Home,/store.html,,,,shop.googlemerchandisestore.com/store.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,True,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,4,23462,8,9,True,NaN,[],[],[],[],[],[],PAGE,web,[],/wishlist.html,shop.googlemerchandisestore.com,Your Wishlist,/wishlist.html,,,,shop.googlemerchandisestore.com/wishlist.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,NaN,0,1,(not set),No,:,(not set),(not set),(not set),(no

In [ ]:
# 첫번째 고유아이디의 hit수 확인
hits_100[hits_100['fullVisitorId']=='309482894121265066']['hitNumber']

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '110', '111', '112', '113', '114', '115', '116', '117', '118',
       '119', '120', '121', '122', '123', '124', '125', '126', '127',
       '128', '129', '130', '131', '132', '133', '134', '135', '136',
       '137', '138', nan], dtype=object)

In [ ]:
# 고유아이디가 알맞게 부여되었음을 확인
hits_100[hits_100['fullVisitorId']=='309482894121265066']['hitNumber'].notna().sum()

138

In [ ]:
# total.hits > 100인 hits컬럼 원본
hits_100.to_csv('hit_100.csv', index=False)

In [ ]:
hits_100.head(2)

,fullVisitorId,visitStartTime,hitNumber,time,hour,minute,isInteraction,isEntrance,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,promotionActionInfo.promoIsView,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,referer,transaction.currencyCode,item.currencyCode,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,contentGroup.contentGroupUniqueViews1,eCommerceAction.option,promotionActionInfo.promoIsClick,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,transaction.transactionId,transaction.transactionRevenue,transaction.transactionTax,transaction.transactionShipping,transaction.affiliation,transaction.localTransactionRevenue,transaction.localTransactionTax,transaction.localTransactionShipping,item.transactionId,transaction.transactionCoupon,isExit
0,3.094829e+17,2017-10-16 15:09:08,1.0,0.0,8.0,9.0,True,True,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,NaN,NaN,NaN,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0.0,True,True,0.0,1.0,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(entrance),(entrance),(entrance),(entrance),(entrance),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.094829e+17,2017-10-16 15:09:08,2.0,7662.0,8.0,9.0,True,NaN,[],[],[],[],[],[],PAGE,web,[],/signin.html,shop.googlemerchandisestore.com,The Google Merchandise Store - Log In,/signin.html,NaN,NaN,NaN,shop.googlemerchandisestore.com/signin.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0.0,True,NaN,0.0,1.0,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**제외컬럼**

In [ ]:
# array(['[]', nan], dtype=object)
hits_100.drop(['customDimensions','customVariables','experiment','customMetrics','publisher_infos'],
              axis=1, inplace=True)

# array([ 0., nan])
hits_100.drop(['appInfo.screenDepth'], axis=1, inplace=True)

# 예외처리코드 array([True, nan], dtype=object)
hits_100.drop(['exceptionInfo.isFatal'], axis=1, inplace=True)

# array([' : ', nan], dtype=object)
hits_100.drop(['social.socialInteractionNetworkAction'], axis=1, inplace=True)

# array(['(not set)', nan], dtype=object)
hits_100.drop(['contentGroup.contentGroup4','contentGroup.contentGroup5'], axis=1, inplace=True)

# 사용통화가 하나뿐이라 미사용 array([nan, 'USD'], dtype=object)
hits_100.drop(['transaction.currencyCode','item.currencyCode'], axis=1, inplace=True)

# 값이 매우 소수 (YouTube:2515 / Google:1683 / Android:188)
hits_100.drop(['contentGroup.contentGroup1'], axis=1, inplace=True)

# 이전방문 페이지 컨텐츠(2를 제외하고 필요성 없을 것으로 예상)
hits_100.drop(['contentGroup.previousContentGroup1','contentGroup.previousContentGroup3',
               'contentGroup.previousContentGroup4','contentGroup.previousContentGroup5'],
              axis=1, inplace=True)

# hits제외 테이블 분석에서 활용
hits_100.drop(['referer'], axis=1, inplace=True)

# 페이지로드 속도 (수집데이터가 적고 분석주제에 불필요)
hits_100.drop(['latencyTracking.pageLoadSample','latencyTracking.pageLoadTime','latencyTracking.pageDownloadTime','latencyTracking.redirectionTime',
               'latencyTracking.speedMetricsSample','latencyTracking.serverResponseTime','latencyTracking.domLatencyMetricsSample',
               'latencyTracking.domInteractiveTime', 'latencyTracking.domContentLoadedTime', 'latencyTracking.domainLookupTime',	'latencyTracking.serverConnectionTime']
               ,axis=1, inplace=True)

# 순매출, 순이익만 사용
hits_100.drop(['transaction.transactionId','item.transactionId','transaction.transactionTax','transaction.localTransactionTax',
               'transaction.transactionShipping','transaction.localTransactionShipping', 'transaction.affiliation'],
               axis=1, inplace=True)		

# 대체가능
hits_100.drop(['page.pagePath', # page.pagePathLevel1로 대체가능
               'page.hostname', # 요소가 2개지만 모두 같은 호스트
               'page.pagePathLevel2','page.pagePathLevel3','page.pagePathLevel4', # 상품경로(다른 세분화된 컬럼 사용)
               'appInfo.screenName', # pagePath와 중복되는 내용
               'social.hasSocialSourceReferral'], # 소셜여부 다른 컬럼으로 확인가능
               axis=1, inplace=True)


**사용컬럼**

In [ ]:
hits_100 = hits_100[['fullVisitorId', 'visitStartTime', 'hitNumber','time','hour','minute',
                    'isInteraction', 'isEntrance', 'isExit',
                    'type', 'dataSource',  # type= event, page, nan, / dataSource=web 또는 nan
                    'page.pagePathLevel1', # level1=세션
                    'appInfo.landingScreenName', 'appInfo.exitScreenName', # 이전페이지, 나간페이지
                    'promotionActionInfo.promoIsView', # 프로모션 확인여부
                    'eCommerceAction.action_type',     # 0: 액션 없음 / 1: 제품 추가 / 2: 제품 삭제 / 3: 제품 조회 / 4: 장바구니 추가 / 5: 결제 진행 / 6: 결제 완료
                    'eCommerceAction.step',  # 1:장바구니, 2:결제정보입력,3:구매완료
                    'social.socialNetwork',  # 소셜미디어방문
                    'contentGroup.contentGroup2', # 카테고리분류
                    'contentGroup.contentGroup3', # 성별 분류
                    'contentGroup.previousContentGroup2',    # 이전 카테고리(카테고리흐름 분석)
                    'contentGroup.contentGroupUniqueViews1', # 방문 깊이1
                    'contentGroup.contentGroupUniqueViews2', # 방문 깊이2
                    'contentGroup.contentGroupUniqueViews3', # 방문 깊이3
                    'eventInfo.eventCategory', # Enhanced Ecommerce : 이벤트대분류 / contact us :문의
                    'eCommerceAction.option',  # 이벤트 중분류 payment, review, Billing and Shipping(배송지입력)
                    'promotionActionInfo.promoIsClick',    # 가격할인 이벤트 사용여부
                    'transaction.transactionCoupon',       # 할인 이름
                    'transaction.transactionRevenue',      # 매출
                    'transaction.localTransactionRevenue', # 순이익(매출에서 비용 제거)
                    'eventInfo.eventAction',               # 액션
                    'eventInfo.eventLabel',                # 선택제품
                    'product', 'promotion'  # json 컬럼(분해필요) 
]]

**이상치를 결측치로 바꿔주기**

In [ ]:
# 'dataSource','social.socialNetwork','contentGroup.contentGroup2','contentGroup.contentGroup3','contentGroup.previousContentGroup2'
# '(not set)'을 nan으로
hits_100.replace('(not set)',np.NaN, inplace=True)

# None을 nan으로
hits_100['transaction.transactionCoupon'].replace('None',np.NaN, inplace=True)

**페이지 값 간소화**

In [ ]:
hits_100['appInfo.landingScreenName'].unique()

array(['shop.googlemerchandisestore.com/home', nan,
       'shop.googlemerchandisestore.com/basket.html',
       'www.googlemerchandisestore.com/home',
       'shop.googlemerchandisestore.com/google+redesign/apparel/men++s/men++s+t+shirts',
       'shop.googlemerchandisestore.com/home-2',
       'shop.googlemerchandisestore.com/google+redesign/apparel/mens/mens+outerwear',
       'shop.googlemerchandisestore.com/google+redesign/bags/totes/chevron+shopper.axd',
       'shop.googlemerchandisestore.com/google+redesign/bags/backpacks',
       'shop.googlemerchandisestore.com/google+redesign/apparel/kid+s/quickview',
       'shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube',
       'shop.googlemerchandisestore.com/store.html',
       'shop.googlemerchandisestore.com/google+redesign/accessories/stickers/home',
       'shop.googlemerchandisestore.com/google+redesign/apparel/kids/kids+toddler',
       'shop.googlemerchandisestore.com/google+redesign/apparel/women+s/women+s

In [ ]:
#  넘어온 페이지/ 현재페이지  / 나간페이지 정리
hits_100['appInfo.landingScreenName'] = hits_100['appInfo.landingScreenName'].str.split('/',expand=True).loc[1]
hits_100['page.pagePathLevel1'] = hits_100['page.pagePathLevel1'].str.split('/', expand=True)[1]
hits_100['appInfo.exitScreenName'] = hits_100['appInfo.exitScreenName'].str.split('/',expand=True)[1]

hits_100[['appInfo.landingScreenName', 'page.pagePathLevel1', 'appInfo.exitScreenName']].head()

,appInfo.landingScreenName,page.pagePathLevel1,appInfo.exitScreenName
0,home,home,google+redesign
1,home,signin.html,google+redesign
2,home,store.html,google+redesign
3,home,wishlist.html,google+redesign
4,home,google+redesign,google+redesign


### hits의 Product컬럼

In [ ]:
# str => json형식으로 바꾸려면 결측치가 없어야하기 때문에 []로 변경해줌
hits_100['product'] = hits_100['product'].fillna('[]')

tqdm.pandas()
product = hits_100['product'].progress_apply(lambda x : ast.literal_eval(x)) # ""을 제거하여 원래 형태로 바꾸기
hits_product = json_normalize(product) # json 값 풀어주기(한 셀에 뭉쳐있는 json을 열별로 풀어줌)
hits_product.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"{'productSKU': '9182722', 'v2ProductName': 'Go...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# product json 해제
emp_product = pd.DataFrame()
product_num = hits_product.columns.max() 
product_id_hit = hits_100[['fullVisitorId','hitNumber']]

for i in tqdm(range(product_num)):
    json_temp = json_normalize( hits_product[i]) # 각 열을 json_normalize
    json_temp = pd.merge(product_id_hit,json_temp,right_index=True,left_index=True)  # 고유id와 값을 이어줌
    json_temp = json_temp.dropna(subset=['productSKU'], axis=0) # 값이 없는 행 제거
    emp_product = pd.concat([emp_product, json_temp],axis=0) # normalize된 값을 붙여주기
emp_product.head()

  0%|          | 0/49 [00:00<?, ?it/s]

c:\Users\ENVY\anaconda3\envs\ts\lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
c:\Users\ENVY\anaconda3\envs\ts\lib\site-packages\IPython\core\formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,fullVisitorId,hitNumber,productSKU,v2ProductName,v2ProductCategory,productVariant,productBrand,productPrice,localProductPrice,isImpression,customDimensions,customMetrics,productListName,productListPosition,productQuantity,productRevenue,localProductRevenue,isClick,productCouponCode
4,3.094829e+17,5.0,9182722,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),(not set),(not set),0,0,True,[],[],Related Products,1,NaN,NaN,NaN,NaN,NaN
5,3.094829e+17,6.0,9182658,Google Women's Short Sleeve Hero Tee Sky Blue,(not set),(not set),(not set),0,0,True,[],[],Related Products,1,NaN,NaN,NaN,NaN,NaN
8,3.094829e+17,9.0,9182723,Google Men's 100% Cotton Short Sleeve Hero Tee...,(not set),(not set),(not set),0,0,True,[],[],Related Products,1,NaN,NaN,NaN,NaN,NaN
9,3.094829e+17,10.0,9182739,Google Men's Watershed Full Zip Hoodie Grey,(not set),(not set),(not set),0,0,True,[],[],Related Products,1,NaN,NaN,NaN,NaN,NaN
10,3.094829e+17,11.0,9182771,Google Women's 1/4 Zip Jacket Charcoal,(not set),(not set),(not set),0,0,True,[],[],Related Products,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516478,5.531986e+18,479.0,GGOEGHGR019499,Google Sunglasses,Lifestyle,RED,(not set),7000000,7000000,NaN,[],[],(not set),0,2,NaN,NaN,NaN,NaN
516479,5.531986e+18,480.0,GGOEGHGR019499,Google Sunglasses,Lifestyle,RED,(not set),7000000,7000000,NaN,[],[],(not set),0,2,NaN,NaN,NaN,NaN
516480,5.531986e+18,481.0,GGOEGHGR019499,Google Sunglasses,Lifestyle,RED,(not set),7000000,7000000,NaN,[],[],(not set),0,NaN,NaN,NaN,NaN,NaN
516481,5.531986e+18,482.0,GGOEGHGR019499,Google Sunglasses,Lifestyle,RED,(not set),7000000,7000000,NaN,[],[],(not set),0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 불필요한 컬럼 제거
emp_product.drop(['productCouponCode', # array([nan, '(not set)'], dtype=object)
                 'productBrand',       # array(['(not set)'], dtype=object)
                 'customDimensions','customMetrics', # 값 없음
                 'productListPosition',# 상품위치(메타데이터 없음)
                 'v2ProductCategory'], # 카테고리 다른 값 사용
                 axis=1, inplace=True) 

In [ ]:
# 보기 쉽게 컬럼 정리
emp_product = emp_product[['fullVisitorId','hitNumber',
                           'v2ProductName',    # 제품이름
                           'productVariant',   # 속성
                           'productListName',  # 관련상품, 카테고리, 검색결과 중 어느곳에서 온건지
                           'productPrice',
                           'localProductPrice',                          
                           'productQuantity', # 수량
                           'productRevenue',  # 매출
                           'localProductRevenue', # 비용제거 수익
                           'isImpression','isClick', # 페이지방문이나 장바구니 수,
                           ]]

# 이상치 제거
emp_product.replace('(not set)', np.NaN, inplace=True)
emp_product.head()

,fullVisitorId,hitNumber,v2ProductName,productVariant,productListName,productPrice,localProductPrice,productQuantity,productRevenue,localProductRevenue,isImpression,isClick
4,3.094829e+17,5.0,Google Men's 100% Cotton Short Sleeve Hero Tee...,NaN,Related Products,0,0,NaN,NaN,NaN,True,NaN
5,3.094829e+17,6.0,Google Women's Short Sleeve Hero Tee Sky Blue,NaN,Related Products,0,0,NaN,NaN,NaN,True,NaN
8,3.094829e+17,9.0,Google Men's 100% Cotton Short Sleeve Hero Tee...,NaN,Related Products,0,0,NaN,NaN,NaN,True,NaN
9,3.094829e+17,10.0,Google Men's Watershed Full Zip Hoodie Grey,NaN,Related Products,0,0,NaN,NaN,NaN,True,NaN
10,3.094829e+17,11.0,Google Women's 1/4 Zip Jacket Charcoal,NaN,Related Products,0,0,NaN,NaN,NaN,True,NaN


In [ ]:
emp_product.to_csv('product.csv', index=False)

In [ ]:
hits_100.drop(['product'], axis=1, inplace=True)

### hits의 Promotion컬럼

In [ ]:
# 결측치는 ast.literal이 적용되지 않아서 [] 값으로 채워줌
hits_100['promotion'] = hits_100['promotion'].fillna('[]')

# 각 값의 ""를 제거하고 본래 type으로 변형
hits_100['promotion'] = hits_100['promotion'].progress_apply(lambda x : ast.literal_eval(x))
promotion = json_normalize(hits_100['promotion'])
promotion.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"{'promoId': 'Apparel Row 1', 'promoName': 'App...","{'promoId': 'Backpacks Row 2 Combo', 'promoNam...","{'promoId': 'Mens T-Shirts Row 3-1', 'promoNam...","{'promoId': 'Womens T-Shirts Row 3-2', 'promoN...","{'promoId': 'Office Row 5 Color Combo', 'promo...","{'promoId': 'Drinkware Row 4 Color Combo', 'pr...","{'promoId': 'Google Brand Row 7-1', 'promoName...","{'promoId': 'YouTube Brand Row 7-2', 'promoNam...","{'promoId': 'Android Brand Row 7-3', 'promoNam...",None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'promoId': 'Apparel Row 1', 'promoName': 'App...","{'promoId': 'Backpacks Row 2 Combo', 'promoNam...","{'promoId': 'Mens T-Shirts Row 3-1', 'promoNam...","{'promoId': 'Womens T-Shirts Row 3-2', 'promoN...","{'promoId': 'Office Row 5 Color Combo', 'promo...","{'promoId': 'Drinkware Row 4 Color Combo', 'pr...","{'promoId': 'Google Brand Row 7-1', 'promoName...","{'promoId': 'YouTube Brand Row 7-2', 'promoNam...","{'promoId': 'Android Brand Row 7-3', 'promoNam...",None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
promotion_num = promotion.columns.max()
emp_promotion = pd.DataFrame()

for i in tqdm(range(promotion_num)):
    hits_promotion = json_normalize(promotion[i])
    hits_promotion = pd.merge(product_id_hit, hits_promotion, right_index=True, left_index=True)
    hits_promotion.dropna(subset=['promoId'], axis=0, inplace=True)
    emp_promotion = pd.concat([emp_promotion, hits_promotion],axis=0)

emp_promotion.head()

  0%|          | 0/12 [00:00<?, ?it/s]

,fullVisitorId,hitNumber,promoId,promoName,promoCreative,promoPosition
0,3.094829e+17,1.0,Apparel Row 1,Apparel,home_main_link_apparel.jpg,Row 1
2,3.094829e+17,3.0,Apparel Row 1,Apparel,home_main_link_apparel.jpg,Row 1
53,3.094829e+17,54.0,Apparel Row 1,Apparel,home_main_link_apparel.jpg,Row 1
500,9.291001e+18,1.0,Apparel Row 1,Apparel,home_main_link_apparel.jpg,Row 1
1007,4.459364e+18,8.0,Apparel Row 1,Apparel,home_main_link_apparel.jpg,Row 1


In [ ]:
emp_promotion.to_csv('promotion.csv',index=False)

In [ ]:
hits_100.drop(['promotion'], axis=1, inplace=True)

In [ ]:
# 결측치만 있는 행 제거 
null_index = hits_100[hits_100.iloc[:,2:4].isnull().all(axis=1)].index
hits_100.drop(null_index, axis=0, inplace=True)
hits_100 = hits_100.reset_index(drop=True)
hits_100.shape

(224199, 32)

In [ ]:
# 이탈페이지 확인
hits_100['appInfo.exitScreenName'].value_counts()

google+redesign                            64517
ordercompleted.html                        47424
basket.html                                38966
store.html                                 10883
home                                       10762
revieworder.html                            8931
myaccount.html?mode=vieworder               7712
myaccount.html?mode=vieworderdetail         7500
payment.html                                7013
store-policies                              5699
yourinfo.html                               3877
signin.html                                 3870
asearch.html                                2071
register.html                               1182
wishlist.html                                957
myaccount.html?mode=billingaddress           859
myaccount.html?mode=giftcertcodes_local      806
myaccount.html?mode=shippingaddress          381
myaccount.html?mode=customerinfo             331
registersuccess.html                         208
myaccount.html?mode=

In [ ]:
# 퍼널분석을 위한 단계설정 컬럼
hits_100['funnel'] = 0

# 이탈페이지 설정
# 홈페이지 방문
hits_100.loc[hits_100['appInfo.exitScreenName']=='home', 'funnel'] = 'home'

# 회원가입
hits_100.loc[hits_100['appInfo.exitScreenName'].isin(['register.html','registersuccess.html']), 'funnel'] = 'signup'

# 로그인
hits_100.loc[hits_100['appInfo.exitScreenName']=='signin.html', 'funnel'] = 'login'

# 상품 페이지
hits_100.loc[hits_100['appInfo.exitScreenName'].isin(['asearch.html', 'google+redesign','store.html','storeitem.html']), 'funnel'] = 'product'

# 결제 페이지
hits_100.loc[hits_100['appInfo.exitScreenName'].isin(['revieworder.html', 'payment.html']), 'funnel'] = 'payment'

# 결제완료
hits_100.loc[hits_100['appInfo.exitScreenName']=='ordercompleted.html', 'funnel'] = 'complete'

In [ ]:
hits_100['funnel'].value_counts()

product     77582
0           67227
complete    47424
payment     15944
home        10762
login        3870
signup       1390
Name: funnel, dtype: int64

In [ ]:
# 퍼널은 완료했지만, 다른 화면에서 이탈한 고객이 있을 경우 방지
order_com = hits_100[hits_100['page.pagePathLevel1'].isin(['ordercompleted.html'])]['fullVisitorId'].unique()

#
hits_100.loc[hits_100['fullVisitorId'].isin(order_com), 'funnel'] = 'complete'

In [ ]:
hits_100['funnel'].value_counts()

complete    92682
product     61530
0           45692
payment     11543
home         8239
login        3225
signup       1288
Name: funnel, dtype: int64

In [ ]:
# tableau에 적합한 데이터형태 완성
hits_100.to_csv('hit_100_ta.csv', index=False)

In [ ]:
hits_100

,fullVisitorId,visitStartTime,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,type,dataSource,page.pagePathLevel1,appInfo.landingScreenName,appInfo.exitScreenName,promotionActionInfo.promoIsView,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.previousContentGroup2,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eCommerceAction.option,promotionActionInfo.promoIsClick,transaction.transactionCoupon,transaction.transactionRevenue,transaction.localTransactionRevenue,eventInfo.eventAction,eventInfo.eventLabel,funnel
0,3.094829e+17,2017-10-16 15:09:08,1.0,0.0,8.0,9.0,True,True,NaN,PAGE,web,home,home,google+redesign,True,0.0,1.0,NaN,NaN,NaN,(entrance),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete
1,3.094829e+17,2017-10-16 15:09:08,2.0,7662.0,8.0,9.0,True,NaN,NaN,PAGE,web,signin.html,home,google+redesign,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete
2,3.094829e+17,2017-10-16 15:09:08,3.0,17130.0,8.0,9.0,True,NaN,NaN,PAGE,web,store.html,home,google+redesign,True,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete
3,3.094829e+17,2017-10-16 15:09:08,4.0,23462.0,8.0,9.0,True,NaN,NaN,PAGE,web,wishlist.html,home,google+redesign,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete
4,3.094829e+17,2017-10-16 15:09:08,5.0,32563.0,8.0,9.0,True,NaN,NaN,PAGE,web,google+redesign,home,google+redesign,NaN,0.0,1.0,NaN,Apparel,Womens,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224194,9.771437e+18,2017-01-04 14:09:57,233.0,3953636.0,7.0,15.0,True,NaN,NaN,PAGE,NaN,yourinfo.html,home,ordercompleted.html,NaN,5.0,1.0,NaN,NaN,NaN,Apparel,NaN,NaN,NaN,NaN,Billing and Shipping,NaN,NaN,NaN,NaN,NaN,NaN,complete
224195,9.771437e+18,2017-01-04 14:09:57,234.0,3964956.0,7.0,16.0,True,NaN,NaN,PAGE,NaN,payment.html,home,ordercompleted.html,NaN,5.0,2.0,NaN,NaN,NaN,Apparel,NaN,NaN,NaN,NaN,Payment,NaN,NaN,NaN,NaN,NaN,NaN,complete
224196,9.771437e+18,2017-01-04 14:09:57,235.0,4020309.0,7.0,16.0,True,NaN,NaN,PAGE,NaN,revieworder.html,home,ordercompleted.html,NaN,5.0,3.0,NaN,NaN,NaN,Apparel,NaN,NaN,NaN,NaN,Review,NaN,NaN,NaN,NaN,NaN,NaN,complete
224197,9.771437e+18,2017-01-04 14:09:57,236.0,4036576.0,7.0,17.0,True,NaN,NaN,PAGE,NaN,ordercompleted.html,home,ordercompleted.html,NaN,6.0,1.0,NaN,NaN,NaN,Apparel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77990000.0,77990000.0,NaN,NaN,complete
